# <h3>Assignment # 2: </h3><h1>Segmenting and Clustering Neighborhoods in Toronto</h1>
<h5>by</h5><h3>Frantzdy Hervé</h3> 
<dl> </dl>
  <dl>
  <dt>Summary:</dt>
  <dd>1. First, we start by creating a new Notebook for this assignment;</dd>
  <dd>2. Second, we use the Notebook to build the code to scrape a given Wikipedia page;</dd>
  <dd>3. Third, create the relevant Dataframe;</dd>
  <dd>4. Finally, we submit a link to our Notebook on our Github repository.</dd>
</dl>
This is the complete workbook which includes both parts of the questions asked from the assignment.

**The following cells will build the code to scrape the given Wikipedia page:** https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: ...working... done

# All requested packages already installed.



In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')

In [4]:
My_table = soup.find('table',{'class':'wikitable sortable'})

**The cells below will create a dataframe following the example provided in the assignment**

In [5]:
#Extracting the rows from the wikipedia table
all_rows = []
for row in My_table.findAll('td'):
    all_rows.append(row.text)

In [6]:
#A list is created out of each element in their given order so each one representing a postal code, a borough, or a neighborhood can be added accordingly
elt = ''
row = []
PC = []
Bo = []
Ne = []

for line in all_rows:
    #elt.append(line.get_text)
    #elt = line.rstrip('\n')   #strip \n the end of each line 
    row.append(line.rstrip('\n'))

PC.append(row[::3])
Bo.append(row[1::3])
Ne.append(row[2::3])


In [7]:
#Checking that the list of lists PC contains only the postal codes extracted
PC[0][0:10]

['M1A', 'M2A', 'M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M8A', 'M9A', 'M1B']

In [8]:
#merging the list into a single table that contains all the three columns created earlier
PC1=PC.pop(0)
Bo1=Bo.pop(0)
Ne1=Ne.pop(0)

col1 = pd.Series(PC1, name='PostalCode')
col2 = pd.Series(Bo1, name='Borough')
col3 = pd.Series(Ne1, name='Neighborhood')

table = pd.concat([col1, col2, col3], axis=1)

In [9]:
#checking the first few rows of the table to observe the occurences of 'Not assigned' values
table.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [10]:
#Filter on the 'Not assigned values'
Neigh_filtr = table[table['Neighborhood'] == 'Not assigned'].index
Neigh_filtr.shape

(77,)

In [11]:
Borough_filtr = table[table['Borough'] == 'Not assigned'].index
Borough_filtr.shape

(77,)

In [12]:
#Apparently, the same postal code that have not been assigned a borough has not been assigned a neighborhood either

In [13]:
index_filt = table[table['Borough']=='Not assigned'].index
table.drop(index_filt, inplace=True)
table.reset_index(drop=True, inplace=True)
table.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
#Confirming that every postal code listed was assigned a neighborhood in the dataframe
Neigh_filtr = table[table['Neighborhood'] == 'Not assigned'].index
Neigh_filtr.shape

(0,)

In [15]:
#Confirming that every postal code listed was assigned a borough in the dataframe
Borough_filtr = table[table['Borough'] == 'Not assigned'].index
Borough_filtr.shape

(0,)

**The cells below will add the geospatial coordinates to the dataframe per associated Postal Code**

In [16]:
#Scraping the geospatial data from the web
import requests # import 

req = requests. get('http://cocl.us/Geospatial_data')
url_content = req.content
csv_file = open('downloaded.csv', 'wb')
csv_file.write(url_content)
csv_file.close()

In [17]:
#Reading the coordinates into a Pandas dataframe
coord = pd.read_csv('downloaded.csv')
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
coord.shape

(103, 3)

In [19]:
#Merging the two tables created in order to the coordinates associated with each postal code in Toronto
final_table = pd.merge(table, coord, right_on='Postal Code', left_on='PostalCode')
final_table.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494


In [20]:
#Since there are two columns with the same the postal code information, we drop one of them
final_table.pop('Postal Code')
final_table.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [21]:
final_table.shape

(103, 5)

**Let's create a superimposed map of the neighborhoods of Toronto with the data we have**

In [22]:
# Required dependecies for a map
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: ...working... done

# All requested packages already installed.



In [23]:
final_table['BoroughDum']=final_table['Borough']
kclusters=len(final_table['BoroughDum'].unique())

In [24]:
final_table['BoroughDum'].replace(to_replace=['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], value=[0,1,2,3,4,5,6,7,8,9],inplace=True)

In [25]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geospatial coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geospatial coordinates of Toronto are 43.6534817, -79.3839347.


In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final_table['Latitude'], final_table['Longitude'], final_table['Borough'], final_table['BoroughDum']):
    label = folium.Popup(str(poi) + ' Neighborhood ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters